In [0]:
# The required imports 
import numpy as np
import pandas as pd
import matplotlib as mlt
import tensorflow as tf
import os
import pathlib
import keras
from keras import regularizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Input
from keras.layers import Activation, ZeroPadding2D, Lambda,Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import layer_utils
from sklearn.utils import class_weight
from keras import backend as K
smooth=1

In [0]:
'''This Function implements the two-way architecture
Input: an array dimension 33x33x4 named img
Output: one hot encoded label for the middle pixel of the input passed (dimension will depend on the Input passed)'''
def two_way_CNN(img):
  
    #The first path (local) of the CNN
  
    O1 = Conv2D(filters= 64,kernel_size =(7,7),strides=(1,1),padding='valid')(img)
    O2 = Conv2D(filters= 64,kernel_size =(7,7),strides=(1,1),padding='valid')(img)
    #Maxout function
    
    Max_O = keras.layers.Maximum()([O1,O2])
    Max_O = MaxPooling2D(pool_size=(4,4), padding='valid',strides=(1,1), data_format='channels_last')(Max_O)
  
    
    O3 = Conv2D(filters= 64,kernel_size =(3,3),strides=(1,1),padding='valid')(Max_O)
    O4 = Conv2D(filters= 64,kernel_size =(3,3),strides=(1,1),padding='valid')(Max_O)
    Max_O = keras.layers.Maximum()([O3,O4])
    Max_O = MaxPooling2D(pool_size=(2,2),padding='valid',strides=(1,1),data_format='channels_last')(Max_O)

    # The second path(global) of the CNN
    O5=Conv2D(filters= 160,kernel_size =(13,13),strides=(1,1),padding='valid')(img)
    O6=Conv2D(filters= 160,kernel_size =(13,13),strides=(1,1),padding='valid')(img)
    Max_O3 = keras.layers.Maximum()([O5,O6])
  
    #Concatenation of both the paths
    Max_O4 = Concatenate()([Max_O,Max_O3])
    #The final output layer
    
    Max_O5 = Conv2D(filters=5,kernel_size=(21,21),strides=(1,1), padding ='valid', activation = 'softmax',kernel_regularizer=regularizers.l1_l2(0.01,0.01))(Max_O4)
    return Max_O5

In [0]:
'''The Local Cascade architecture is implemented here.
Inputs:None
Output:An instance of model class of keras API which implements the Local Cascade CNN'''
def localCascadeCNN():
  #img1 is of size 56x56x4 is passed through two way CNN
  #img2 is of size 33x33x4
  img1 = Input((56,56,4))
  #Output of the first CNN
  O1=two_way_CNN(img1)
  img2 = Input((33,33,4))
 #The first path of the second CNN
  O2=Conv2D(filters= 69,kernel_size =(7,7),strides=(1,1),padding='valid')(img2)
  O3=Conv2D(filters= 69,kernel_size =(7,7),strides=(1,1),padding='valid')(img2)
  Max_O=keras.layers.Maximum()([O2,O3])
  Max_O = MaxPooling2D(pool_size=(4,4),padding='valid',strides=(1,1),data_format='channels_last')(Max_O)
  #the output of first CNN is concatenated to the hidden layer
  O4=Concatenate()([O1,Max_O])

  O5 = Conv2D(filters= 64,kernel_size =(3,3),strides=(1,1),padding='valid')(O4)
  O6 = Conv2D(filters= 64,kernel_size =(3,3),strides=(1,1),padding='valid')(O4)
  Max_O1 = keras.layers.Maximum()([O5,O6])
  Max_O1 = MaxPooling2D(pool_size=(2,2),padding='valid',strides=(1,1),data_format='channels_last')(Max_O1)
  #The second path of the second CNN
  O7=Conv2D(filters= 160,kernel_size =(13,13),strides=(1,1),padding='valid')(img2)
  O8=Conv2D(filters= 160,kernel_size =(13,13),strides=(1,1),padding='valid')(img2)
  Max_O2 = keras.layers.Maximum()([O7,O8])
  
  Max_O3 = Concatenate()([Max_O1,Max_O2])
  #the final output
  Max_O4 = Conv2D(filters=5,kernel_size=(21,21),strides=(1,1), padding ='valid', activation = 'softmax')(Max_O3)
  final_model = Model(inputs = [img1, img2], outputs = Max_O4 )
  return final_model

In [0]:
#Initialization of the model and its summary
model2 = localCascadeCNN()
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 56, 56, 4)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 50, 50, 64)   12608       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 50, 50, 64)   12608       input_1[0][0]                    
__________________________________________________________________________________________________
maximum_1 (Maximum)             (None, 50, 50, 64)   0           conv2d_1[0][0]                   
                                                                 conv2d_2[0][0]             

In [0]:

!pip3 install medpy

In [0]:

''' Function for processing the MRI images
Input : paths of all 4 image modalities and the ground truth 
Output : two lists of size 155, one containing processed input of size 240x240x4 stored as n-dimensional numpy array
         the other containing ground truth of dimension 240x240x155 stored as n-dimensional numpy array'''
from sklearn.preprocessing import normalize
from PIL import Image
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from medpy.io import load, save
def gen_image(path1, path2, path3,path4,gtpath):   
  image_t1, image_header1 = load(path2)
  image_t2, image_header2 =load(path4)
  image_t1c, image_header1c =load(path3)
  image_flair, image_headerr =load(path1)
  image_gt, image_header_gt = load(gtpath)
  t1x, t1y, t1z = image_t1.shape
  final = np.zeros((155, 240, 240,4))
#for extracting each axial slice from each modality and normalising it
  for a in range(0, 155):
    imgt1 = image_t1[:,:,a]
    imgt1 = normalize(imgt1)
    imgt2 = image_t2[:,:,a]
    imgt2 = normalize(imgt2)
    imgt1c = image_t1c[:,:,a]
    imgt1c = normalize(imgt1c)
    imgtflair = image_flair[:,:,a]
    imgtflair = normalize(imgtflair)
  #for stacking modalites depth wise and storing them in the list
    final[a] = np.dstack([imgt1, imgt2, imgt1c, imgtflair])


  return final, image_gt


In [0]:
'''This function creates a balanced dataset of all the labels. Each label will contain at most 2000 instances.
Input:Inputs : list of inputs of one brain(array), the ground truth numpy array(gt)
Outputs: a list of patch sized 56x56x4, a list of patch sized 33x33x4, a list of one hot encoded ground truth and a count of each lable'''
def create_patch(array, gt):
    patches56=[]
    patches33=[]
    
    
    gt1=np.zeros((10000,1,1,5))
    actual_gt=np.zeros(10000,dtype=int)
    count0,count1,count2,count3,count4=0,0,0,0,0
    k=0
    
    for slice in range(30,130):
        c=30
        for i in range(0,80):
            r=30
            for j in range(0,80):
              
                

                key=gt[r+32,c+32,slice]
                
                actual_gt[k]=key
                
                
                
                if(key==2 and count2<2000 ):
                  patch=array[slice,r:r+56,c:c+56,:]
                  s_patch=patch[16:49,16:49,:]
                  patches56.append(patch)
                  patches33.append(s_patch)
                  gt1[k,0,0,key]=1
                  k=k+1
                  count2 = count2+1
                  if(key==1 and count1<2000 ):
                  patch=array[slice,r:r+56,c:c+56,:]
                  s_patch=patch[16:49,16:49,:]
                  patches56.append(patch)
                  patches33.append(s_patch)
                  gt1[k,0,0,key]=1
                  k=k+1
                  count1 = count1+1
                if(key==3 and count3<2000 ):
                  patch=array[slice,r:r+56,c:c+56,:]
                  s_patch=patch[16:49,16:49,:]
                  patches56.append(patch)
                  patches33.append(s_patch)
                  gt1[k,0,0,key]=1
                  k=k+1
                  count3 = count3+1
                if(key==4 and count4<2000 ):
                  patch=array[slice,r:r+56,c:c+56,:]
                  s_patch=patch[16:49,16:49,:]
                  patches56.append(patch)
                  patches33.append(s_patch)
                  gt1[k,0,0,key]=1
                  k=k+1
                  count4 = count4+1
                r=r+2
            c=c+2
    for slice in range(50,130):
        l=30
        for i in range(0,6):
            m=30
            for j in range(0,6):
              key=gt[l+32,m+32,slice]
              if(k==8000 or count0==1999):
                gt2 = gt1[0:k]
                return patches56, patches33, gt2, actual_gt,count0,count2,count3,count4
              actual_gt[k]=key

              if(key==0 and count0<2000 ):
                patch=array[slice,l:l+56,m:m+56,:]
                s_patch=patch[16:49,16:49,:]
                patches56.append(patch)
                patches33.append(s_patch)
                gt1[k,0,0,key]=1
                k=k+1
                count0 = count0+1
              m=m+20
            l=l+20
    gt2 = gt1[0:k]
    
    return patches56, patches33, gt2, actual_gt, count0,count2,count3,count4

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
'''This cell trains the model. Change the path as per the requirement'''
from sklearn.utils import class_weight
from keras import backend as K
smooth=1
#path = '/content/gdrive/My Drive/NNFL Assignment/BRATS 2015 Dataset/Training'
path='/content/gdrive/My Drive/NNFL/DATASET/BRATS2015_Training'
#path='/content/gdrive/My Drive/BRATS2015_Training'
j=0
model=localCascadeCNN()
with os.scandir(path) as training:
    for folder1 in training:
        path1 = path + '//' + folder1.name
        with os.scandir(path1) as lgg_hgg:
            for folder2 in lgg_hgg:
                path2 = path1 + '//' + folder2.name
                with os.scandir(path2) as brats:
                    modularity_path = []
                    for folder3 in brats:          
                        path3 = path2 + '/' + folder3.name
                        with os.scandir(path3) as vsdbrain:
                            for file in vsdbrain:
                                if file.name.endswith('mha'):
                                    path4 = path3 + '/' + file.name
                                    modularity_path.append(path4)      
                
                sorted_path = sorted(modularity_path)
                
                try:
                  arr, gt= gen_image(sorted_path[0], sorted_path[1], sorted_path[2], sorted_path[3], sorted_path[4])
                  patches65, patches33, gt_pixel, actual_gt,x, a,b,c,d = create_patch(arr, gt)
                  print(x,a,b,c,d)
                  
                  #to account for possible unbalanced data..
                  #class_weight: useful to tell the model to "pay more attention" to samples from an under-represented class.
                  
                  class_weights = class_weight.compute_class_weight('balanced',np.unique(actual_gt),actual_gt)
                  
                  #for regular saving of the model
                  if(j==5 or j==35 or j==75 or j==100 or j==150 or j==200 or j==250 or j==270):
                    model_json = model.to_json()
                    with open("/content/gdrive/My Drive/model2.json","w") as json_file:
                      json_file.write(model_json)
                    model.save_weights("/content/gdrive/My Drive/weight274.h5")
                    print("Saved model to disk")

                  #Compilation and Training of the model                      
                  sgd = keras.optimizers.SGD(learning_rate=0.005, momentum=0.5, nesterov=False)
                  model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.Precision(class_id=0, name = 'normal_tissue_precision'),tf.keras.metrics.Precision(class_id=2,name = 'edema_precision'),tf.keras.metrics.Precision(class_id=3,name = 'non_enhancing_tumor_precision'),tf.keras.metrics.Precision(class_id=4,name = 'enhancing_tumor_precision'),tf.keras.metrics.Recall(class_id=0,name = 'normal_tissue_recall'),tf.keras.metrics.Recall(class_id=2,name = 'edema_recall'),tf.keras.metrics.Recall(class_id=3,name = 'non_enhancing_tumor_recall'),tf.keras.metrics.Recall(class_id=4,name = 'enhancing_tumor_recall')])
                  model.fit([patches65, patches33],gt_pixel,epochs=1, class_weight=class_weights)
                  j=j+1
                  print(j)
                except:
                  print(j)